In [32]:
import pandas as pd
from numpy import nan

In [40]:
class setup_data():
    def __init__(self, file):
        self.file = file
        self.sheets = ['Education','Early_childhood_development','Families', 'Housing','IRSD', 'Mothers_babies']
        self.data = self.create_data()
        self.dataset = self.merge_data()

    def init_sheet(self,sheet): # function to read individual sheets from excel
        df = pd.read_excel(self.file, sheet_name = sheet)
        df.columns = df.iloc[3]
        df = df.iloc[4:596]
        df = df.dropna(axis = 'columns')
        df = df.rename(columns = {'Code\n(PHN/LGA)': 'LGA_code'})
        df = df.replace('..', nan)
        df = df.replace('#', nan)
        return df

    def create_data(self): # make each sheet into its own data frame
        data = {}
        for sheet in self.sheets: # loop to add each dataframe as a value in a dictionary, so it can be called 
            data[f'{sheet}'] = self.init_sheet(sheet)

        data['ED_total'] = self.ED_data()
        data['Hosp_ad'] = self.hosp_data()
        data['Income_support'] = self.income_support()
        return data
         
        
    def merge_data(self):
        dataset = pd.DataFrame()
        for data in self.data.keys():
            dataset = pd.concat([dataset, self.data[data]], axis =1)

        dataset = dataset.drop(columns=['Quality indicator*','Name\n(PHN/LGA)'])
        return dataset
    
    def ED_data(self):
        ED = pd.read_excel(self.file, sheet_name = 'ED_total')
        ED_col = ED.loc[:, ED.columns.str.contains('Emergency')].columns.to_list()# save the column titles
        # making column titles easier to read
        ED_col = [w.replace("Emergency department presentations: Total presentations for " , "") for w in ED_col]
        ED_col = [w.replace("\n", "") for w in ED_col] # delete linebreaks
        ED_col = [w.replace("-", "") for w in ED_col] # delete hyphens
        ED_col = [w.replace("  ", " ") for w in ED_col] # reduce double whitespace to single
        ED_col.insert(0, 'LGA_code')

        ED.columns = ED.iloc[3]
        ED = ED.iloc[4:596]
        ED = ED.rename(columns = {'Code\n(PHN/LGA)': 'LGA_code'})
        ED = ED.drop(columns=['Quality indicator*', 'Name\n(PHN/LGA)','Number','SR'])# remove all except ASR
        ED = ED.dropna(axis = 'columns')

        ED = ED.replace('..', nan)
        ED = ED.replace('#', nan)

        ED.columns = ED_col# replace column titles

        return ED

    def hosp_data(self):
        hosp = pd.read_excel(self.file, sheet_name = 'Admiss_principal_diag_persons')
        hosp_col = hosp.loc[:, hosp.columns.str.contains('Admissions for ')].columns.to_list()# save the column titles
        # making column titles easier to read
        hosp_col = [w.replace("Admissions for " , "") for w in hosp_col]
        hosp_col = [w.replace("\n", "") for w in hosp_col]
        hosp_col = [w.replace("-", "") for w in hosp_col]
        hosp_col = [w.replace("  ", " ") for w in hosp_col]
        hosp_col.insert(0, 'LGA_code')

        hosp.columns = hosp.iloc[3]
        hosp = hosp.iloc[4:596]
        hosp = hosp.rename(columns = {'Code\n(PHN/LGA)': 'LGA_code'})
        hosp = hosp.drop(columns=['Quality indicator*', 'Name\n(PHN/LGA)','Number','SR'])# remove all except ASR
        hosp = hosp.dropna(axis = 'columns')
        hosp = hosp.replace('..', nan)
        hosp = hosp.replace('#', nan)

        hosp.columns = hosp_col# replace column titles
        return hosp
    
    def income_support(self):
        income = self.init_sheet('Income_support')
        income = income[['% young people receiving Youth Allowance (other)', 
                         '% low income, welfare-dependent families (with children)',
                         '% children in low income, welfare-dependent families']]

        return income        
        




In [3]:
file = '2021_data.xlsx'


In [41]:
test_data = setup_data(file)


In [42]:
test_data.dataset

,LGA_code,Children aged 4 years old enrolled in a preschool program,Children aged 4 years old,% children aged 4 years old enrolled in a preschool program,Children aged 5 years old enrolled in a preschool program,Children aged 5 years old,% children aged 5 years old enrolled in a preschool program,Children aged 4 and 5 years old enrolled in a preschool program,Children aged 4 and 5 years old,% children aged 4 and 5 years old enrolled in a preschool program,...,"musculoskeletal system and connective tissue diseases, 15 to 24 years Public hospitals","genitourinary system diseases, 0 to 14 years Public hospitals","genitourinary system diseases, 15 to 24 years Public hospitals","congenital malformations, deformations and chromosomal abnormalities, 0 to 14 years Public hospitals","congenital malformations, deformations and chromosomal abnormalities, 15 to 24 years Public hospitals","injury, poisoning and other external causes, 0 to 14 years Public hospitals","injury, poisoning and other external causes, 15 to 24 years Public hospitals",% young people receiving Youth Allowance (other),"% low income, welfare-dependent families (with children)","% children in low income, welfare-dependent families"
4,PHN101,13298.547814,17669.969523,75.260729,2997.275836,16431.259217,18.241303,16295.82365,34101.22874,47.786617,...,198.166466,256.098610,412.841903,366.292244,38.844650,947.581165,1231.815331,2.066609,2.737056,6.542618
5,11100,477.33213,642.273645,74.319121,84.360033,583.742739,14.451577,561.692164,1226.016384,45.814409,...,780.656874,172.659286,617.136936,411.556019,92.660483,957.038444,2019.826827,3.285671,3.411211,8.119375
6,11300,241,353.109432,68.250797,52,379.244615,13.711467,293,732.354047,40.007972,...,72.276469,313.407607,224.108524,325.380743,80.219550,885.416016,753.310047,1.221519,2.613780,7.015772
7,11520,835,1031.014288,80.988208,162,990.244132,16.359602,997,2021.25842,49.325707,...,170.364697,178.773906,409.955935,372.006280,NaN,906.211972,1143.793893,1.292710,1.544451,3.675856
8,11570a,1632.322182,2433.445813,67.078633,251.358646,2241.251354,11.215103,1883.680828,4674.697167,40.295248,...,338.525447,278.788151,565.249634,420.310732,65.031603,1086.269986,1512.227180,4.219953,8.045179,17.596662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,74660,21,87.359212,24.038678,4,120.62125,3.316165,25,207.980462,12.02036,...,532.370495,365.255644,646.017661,NaN,NaN,2813.471473,4475.658801,32.650814,20.624049,23.041432
592,74680,5,63.279943,7.901398,0,93.099558,0,5,156.379501,3.19735,...,NaN,NaN,NaN,NaN,NaN,2076.817925,3789.556529,45.420995,26.072607,22.801921
593,79399,75,97.34122,77.048552,10,124.434616,8.036349,85,221.775835,38.326989,...,825.959675,717.327852,2509.838472,485.481695,NaN,4083.223167,8424.383827,11.149348,5.567154,8.283398
594,PHN801,5409,5864.0,92.240791,1632,5948.0,27.437794,7041,11812.0,59.608872,...,377.798183,307.633277,801.936143,530.022478,68.283800,1611.361457,2763.972980,3.067021,3.046611,6.745388
